# Zukunftsvorhersage

Dieses Notebook enthält den Code, um Vorhersagen für die Wohnbevölkerung der Gemeinden des Kanton Luzerns zu machen. Das Trainieren des Models benötig 1 bis 2 Minuten.

### Vorhersage

Im vorangegangenen Notebook wurden die Ergebnisse des Modells anhand der Testdaten diskutiert. Das eigentliche Ziel besteht jedoch darin, das Modell zur Vorhersage zukünftiger Jahre einzusetzen. Um dieses Szenario zu simulieren, wurde das Modell erneut trainiert, wobei sämtliche verfügbaren Daten verwendet wurden, ohne dabei die Hyperparameter anzupassen. Anschliessend wurden mithilfe dieses Modells Vorhersagen für die Bevölkerungszahlen der Gemeinden im Kanton Luzern für die kommenden acht Jahre generiert. Natürlich lässt sich die Genauigkeit dieser Prognosen nicht direkt bewerten. Dennoch war es äusserst interessant, einen näheren Blick auf die einzelnen Prognosen zu werfen. Insbesondere fiel auf, dass die zuvor erwähnten "Problem-Gemeinden" nun konsistente und plausible Vorschläge lieferten, ohne ungewöhnliche Sprünge. Die genaue Ursache dafür, dass das Modell nun eine bessere Leistung zu zeigen schien, konnten wir letztendlich nicht eindeutig erklären. Eine Hypothese besteht darin, dass die grössere Anzahl an Datenpunkten zu einer Stabilisierung der Prediction beigetragen haben könnte. 

### Requirements

Dieses Notebook benötigt das Excel 'Alle_Daten'.

#### Python Version

Python version used in this notebook: Python 3.10.2

#### Libraries

In [ ]:
!pip install scikit-learn
!pip install pandas
!pip install matplotlib

### Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import pickle
import ipywidgets as widgets

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

### Funktionen

Datensatz von Excel einlesen

In [ ]:
# function to read in the data
def read_excel(target):
    column_names_to_load = [
        'Siedlungsfläche_in_%', 
        'Landwirtschafts-fläche_in_%',
        'Betriebe_total',
        'Wohnungen - Total',
        'Anzahl_Privathaushalte',
        'Gemeindename',
        'Gemeindetypologien',
        target]

    # Load all data from excel
    path = 'Data/Preparation/Kennzahlen_aller_Gemeinden/Alle_Daten.xlsx'
    data = pd.read_excel(path, header=[0,1], index_col=[0,1])

    # Extract wanted columns
    column_mask = data.columns.isin(column_names_to_load, level=1)
    data = data.loc[:, column_mask]

    # Drop top level column names
    data = data.droplevel(0,axis=1)

    # Reset index
    data = data.reset_index()

    return data

Splitfunktion

In [ ]:
def train_test_split(data, test_length):
    start_year = 1991
    amount_of_years = 38

    # Define length of splits
    train_length = amount_of_years - test_length

    # Define years where train and validation split ends
    train_data_end_year = start_year + train_length

    # Apply splits
    train = data[data['Jahr'] <= train_data_end_year]
    test = data[data['Jahr'] > train_data_end_year]

    
    return train, test

Definiere x und y

In [ ]:
def x_y_split(data, target):
    y_column_name = target

    x_column_names = [
        'Jahr',
        'Siedlungsfläche_in_%', 
        'Landwirtschafts-fläche_in_%',
        'Betriebe_total',
        'Wohnungen - Total',
        'Anzahl_Privathaushalte',
        'Gemeindename',
        'Gemeindetypologien'
        ]

    # Execute the Y split
    y = data[y_column_name]
    y.name = "y"

    x = data[x_column_names]

    assert len(x) == len(y), 'X and Y split need to have the same length'

    return x, y

Dummyvariablen definieren und Skalierung

In [ ]:
def set_dummy_variables(x):
    # Set dummie variables for the column Gemeindetypologien and Gemeindename in the X split
    return pd.get_dummies(x, columns=['Gemeindetypologien', 'Gemeindename'])

def scale(x, scaler):
    # Store the column names that it can be reapplied after scaling
    x_columns = x.columns

    # Scale the data with the StandardScaler
    if not scaler:
        scaler = StandardScaler()
        x = scaler.fit_transform(x)
    else:
        x = scaler.transform(x)
    
    # Create pandas dataframes from ndArrays and reapply columnnames
    x = pd.DataFrame(x, columns=x_columns)

    return x, scaler

Daten für die Visualisierungen vorbereiten

In [ ]:
def get_jahre(x):
    # All years from the x set
    return list(x['Jahr'].unique())

def get_gemeinde_and_gemeindetypologien(x):
    # All Gemeinde dummy variable column names
    gemeinden = x.columns.tolist()[11:]

    # All Gemeindetypologien dummy variable column names
    gemeinden_typologien = x.columns.tolist()[6:10]

    return gemeinden, gemeinden_typologien

Einzelne Gemeinde ploten

In [ ]:
def plot_gemeinde(gemeinde, jahre_train, jahre_test, x_list, y_list, y_pred_list):    
    # Concat the train and validation set for the X data set
    x = pd.concat(x_list, axis=0).reset_index()

    # Concat the train and validation set for the Y data set
    y = pd.concat(y_list, axis=0).reset_index()
    
    # Concat the predicted values from the train and validation set
    y_pred = pd.concat(y_pred_list, axis=0).reset_index()

    # Concat the X, Y, and the predicted values
    visualisation_all_gemeinde_data = pd.concat([pd.DataFrame(x), y, y_pred], axis=1)

    # Extract the data for one gemeinde. Get the rows where the gemeinde dummy variable is positive
    visualisation_gemeinde_data = visualisation_all_gemeinde_data[visualisation_all_gemeinde_data[gemeinde] > 0]

    # Extract the expected and the predicted value
    gemeinde_data_y = visualisation_gemeinde_data[0]
    gemeinde_pred = visualisation_gemeinde_data['pred']

    # Plot the data
    plt.figure()

    # Set title on plot
    plt.title(gemeinde)
    
    # plot the expected data as dots
    plt.scatter(jahre_train, gemeinde_data_y[:len(jahre_train)], label='Train')

    # plot the prediction as line
    plt.plot(jahre_train + jahre_test, gemeinde_pred, color="black", label='Prediction')

    # Show legend
    plt.legend()

    # activate this for fixed y-axis
    # plt.ylim([50, 2500])

    plt.show()

    # Save the image and close the plot that it doesn't display in the jupyter notebook
    #plt.savefig(f'{folder}/Gemeinden/{gemeinde}.png')
    #plt.close()
      


### Datenaufbereiten

In [ ]:
# to switch target you can swicht the lines
target = "Ständige Wohnbevölkerung Total"
# target ='Ständige Wohnbevölkerung Bevölkerungs-dichte1 in Pers./km2'

# Load data if not already loaded
data = read_excel(target)

# x_y_split
x_data, y_data = x_y_split(data, target)

Simulierung des Datensatzes für die Zukunftsvorhersage

In [ ]:
# simulate future data
# define year to predict into the future
jahre_zukunft = [2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029]
temp = x_data[x_data['Jahr']==2021]
gemeinde_namen = temp.Gemeindename.unique().tolist()

df_list = []
for j in gemeinde_namen:
    i = temp[temp['Gemeindename']== j ]
    # repeat the first row 8 times
    repeated_row = pd.concat([i.iloc[0:1]] * 8, ignore_index=True)
    # change the year entries
    repeated_row.loc[:, 'Jahr'] = jahre_zukunft

    # append the repeated row to the dataframe
    #df = pd.concat([temp, repeated_row], ignore_index=True)
    df_list.append(repeated_row )

# combine to one dataframe
df_list.append(x_data)
x = pd.concat(df_list, ignore_index=True)


Daten vorbereiten für das Model

In [ ]:
# Define training and test length
TRAIN_LENGTH = 30
TEST_LENGTH = 8

# Split
x_data, x_future = train_test_split(x, TEST_LENGTH)

# Set dummy variables
x_data = set_dummy_variables(x_data)
x_future = set_dummy_variables(x_future)

# Extract data for visualisations before scaling
jahre = get_jahre(x_data)
jahre_zukunft = get_jahre(x_future)
gemeinden, gemeinden_typologien = get_gemeinde_and_gemeindetypologien(x_data)

# Scale the data
x_data, scaler = scale(x_data, None)
x_future, scaler = scale(x_future, scaler)

### Model trainieren

In [ ]:
# define model and model name
model_name = f'T{TRAIN_LENGTH} V{TEST_LENGTH} SVR POLY3 C=80 Gamma=0.015'
model = SVR(kernel='poly', degree=3, C=30, gamma=0.1)

print("Train model:", model_name)

# Train Model
model.fit(x_data, y_data)

# or load already stored weights if desired
# with open(f'zukunfst_model_weights.pkl', 'rb') as f:
#     model = pickle.load(f)

# Save the trained model weights
with open(f'zukunfst_model_weights.pkl', 'wb') as f:
    pickle.dump(model, f)


# Prediction
y_pred = pd.Series(model.predict(x_data), name = 'pred')
y_pred_future = pd.Series(model.predict(x_future), name = 'pred')
        

### Bevölkerungsvorhersagen mit trainiertem Model

In [ ]:
# predict the values of a gemeinde
def prediction_for_single_gemeinde(Gemeinde):
    key = Gemeinde
    single_prediciton = model.predict(x_future[x_future[key] > 0]).tolist()
    paired = list(zip(jahre_zukunft, single_prediciton))
    print (Gemeinde)
    for i in paired:
        print(i)


widgets.interact(prediction_for_single_gemeinde, Gemeinde=widgets.Dropdown(
    options=gemeinden
));

In [ ]:
def call_plots(Gemeinde):
    plot_gemeinde(Gemeinde, 
    jahre, 
    jahre_zukunft,
    [x_data, x_future], 
    [y_data, y_pred_future], 
    [y_pred, y_pred_future]
    )


widgets.interact(call_plots, Gemeinde=widgets.Dropdown(
    options=gemeinden
));